In [1]:
# Install packages (quietly)
!pip -q install pandas numpy matplotlib seaborn statsmodels patsy pyjanitor tqdm

import os, re, io, warnings, numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns
warnings.filterwarnings("ignore")
plt.rcParams["figure.dpi"] = 120

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.discrete.discrete_model import NegativeBinomial as NB2
from patsy import dmatrices
from patsy import bs
from tqdm import tqdm
tqdm.pandas()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.4/215.4 kB 3.8 MB/s eta 0:00:00


In [2]:
from google.colab import files
from zipfile import ZipFile

uploaded = files.upload()        # choose your ZIP (e.g., "ap stat dataset.zip")
zip_name = list(uploaded.keys())[0]
print("Uploaded:", zip_name)

os.makedirs("/content/data", exist_ok=True)
with ZipFile(io.BytesIO(uploaded[zip_name])) as zf:
    zf.extractall("/content/data")

# Show extracted files
for root, _, files_ in os.walk("/content/data"):
    for f in files_:
        if f.lower().endswith((".csv", ".xlsx", ".txt")):
            print(os.path.join(root, f))

os.makedirs("/content/outputs", exist_ok=True)


Saving ap stat dataset.zip to ap stat dataset.zip
Uploaded: ap stat dataset.zip
/content/data/pollutants_and_hosp.csv


In [3]:
# Find first CSV (edit if you want a specific one)
csv_candidates = []
for root, _, files_ in os.walk("/content/data"):
    for f in files_:
        if f.lower().endswith(".csv"):
            csv_candidates.append(os.path.join(root, f))

assert len(csv_candidates) > 0, "No CSV found inside the ZIP. Please check the archive."
CSV_PATH = csv_candidates[0]
print("Using CSV:", CSV_PATH)

# Read with a separator guess
for sep in [",",";","\t","|"]:
    try:
        df_raw = pd.read_csv(CSV_PATH, sep=sep, engine="python")
        if df_raw.shape[1] > 3:
            break
    except Exception:
        pass

df = df_raw.copy()

# Clean names: lower, underscores, strip punctuation
df.columns = (
    df.columns.str.strip()
              .str.lower()
              .str.replace(r"[^\w]+", "_", regex=True)
              .str.replace("__+", "_", regex=True)
              .str.strip("_")
)

print("Shape:", df.shape)
df.head(3)


Using CSV: /content/data/pollutants_and_hosp.csv
Shape: (1462, 21)


,data,sih_sul_i_old,sih_sul_j_old,sih_sul_i_kid,sih_sul_j_kid,sih_gv_i_old,sih_gv_j_old,sih_gv_i_kid,sih_gv_j_kid,pm25_gv,...,so2_gv,no2_gv,co_gv,o3_gv,co_sul,o3_sul,no2_sul,so2_sul,pm10_sul,pm25_sul
0,01/01/2015,0,3,0,1,11,5,1,9,11.680254,...,10.667604,7.143889,296.19325,31.951042,298.583542,35.724583,9.857808,25.510417,27.001052,5.857146
1,02/01/2015,0,1,0,1,10,6,0,8,12.645833,...,10.544443,9.440139,341.62775,30.016771,372.735464,36.766667,15.922458,37.404937,34.650446,9.369452
2,03/01/2015,1,2,0,1,2,2,1,2,11.020833,...,9.020417,9.303472,344.04400,30.358958,327.966098,34.597917,12.472983,26.540933,33.638609,8.496304


In [4]:
# --- Date detection ---
date_candidates = [c for c in df.columns if c in ["date","data","dt","day","day_date"] or re.search(r"^(date|data)", c)]
assert len(date_candidates) > 0, f"Couldn't auto-detect a date column. Found: {df.columns.tolist()[:20]}"
DATE_COL = date_candidates[0]

def try_parse_date(s):
    for fmt in ("%Y-%m-%d","%d/%m/%Y","%d-%m-%Y","%m/%d/%Y"):
        try:
            return pd.to_datetime(s, format=fmt, errors="raise")
        except Exception:
            pass
    return pd.to_datetime(s, errors="coerce")

df["date"] = try_parse_date(df[DATE_COL])
assert df["date"].notna().any(), "Date parsing failed—please set DATE_COL or add a format."
df = df.sort_values("date").reset_index(drop=True)

# --- Guess outcomes & pollutants ---
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()

# Outcomes (daily counts) heuristic
guess_outcomes = [c for c in num_cols if re.search(r"(sih|hospital|admi|resp|icd|cases|count)", c)]
if len(guess_outcomes) > 12:
    means = df[guess_outcomes].mean().sort_values()
    guess_outcomes = means.index.tolist()[:12]

# Pollutants
pollutant_keys = ["pm25","pm10","no2","so2","co","o3"]
guess_pollutants = [c for c in num_cols if any(k in c for k in pollutant_keys)]

# Region suffixes (e.g., _gv, _sul)
suffixes = set()
for c in guess_outcomes + guess_pollutants:
    m = re.search(r"_(gv|sul|north|south|east|west)$", c)
    if m: suffixes.add(m.group(1))
suffixes = list(suffixes) if suffixes else ["gv","sul"]

print("Guessed DATE_COL:", DATE_COL)
print("Guessed OUTCOMES:", guess_outcomes)
print("Guessed POLLUTANTS:", guess_pollutants)
print("Detected/assumed region suffixes:", suffixes)

# ---- EDIT below if needed ----
OUTCOMES   = guess_outcomes      # e.g., ["sih_gv_j_kid", "sih_gv_j_old", ...]
POLLUTANTS = guess_pollutants    # e.g., ["no2_gv","co_gv","pm10_gv", "no2_sul", ...]
REGION_SUFFIX_PRIORITY = ["gv","sul"]
# --------------------------------

# Time helpers
df["dow"]  = df["date"].dt.day_name().str[:3]
df["year"] = df["date"].dt.year
df["t"]    = (df["date"] - df["date"].min()).dt.days.astype(int)

print("Final OUTCOMES:", OUTCOMES)
print("Final POLLUTANTS:", POLLUTANTS)


Guessed DATE_COL: data
Guessed OUTCOMES: ['sih_sul_i_old', 'sih_sul_j_old', 'sih_sul_i_kid', 'sih_sul_j_kid', 'sih_gv_i_old', 'sih_gv_j_old', 'sih_gv_i_kid', 'sih_gv_j_kid']
Guessed POLLUTANTS: ['pm25_gv', 'pm10_gv', 'so2_gv', 'no2_gv', 'co_gv', 'o3_gv', 'co_sul', 'o3_sul', 'no2_sul', 'so2_sul', 'pm10_sul', 'pm25_sul']
Detected/assumed region suffixes: ['sul', 'gv']
Final OUTCOMES: ['sih_sul_i_old', 'sih_sul_j_old', 'sih_sul_i_kid', 'sih_sul_j_kid', 'sih_gv_i_old', 'sih_gv_j_old', 'sih_gv_i_kid', 'sih_gv_j_kid']
Final POLLUTANTS: ['pm25_gv', 'pm10_gv', 'so2_gv', 'no2_gv', 'co_gv', 'o3_gv', 'co_sul', 'o3_sul', 'no2_sul', 'so2_sul', 'pm10_sul', 'pm25_sul']


In [5]:
# Table 1: Descriptive statistics (save for report)
desc = df[sorted(set(OUTCOMES + POLLUTANTS))].describe().T
desc.to_csv("/content/outputs/descriptive_stats.csv")
desc.head(10)


,count,mean,std,min,25%,50%,75%,max
co_gv,1462.0,398.980719,87.866808,189.541667,337.502458,389.209333,451.858792,752.367232
co_sul,1445.0,303.176184,95.516844,85.983000,242.430000,293.034583,350.116042,1926.607986
no2_gv,1451.0,16.822951,6.602466,4.425000,11.735208,15.605833,21.491267,41.885652
no2_sul,1459.0,6.697719,3.037750,1.425639,4.431506,6.188553,8.409635,20.101683
o3_gv,1462.0,29.484663,9.541588,9.208472,22.399358,28.317708,35.065486,78.576250
o3_sul,1109.0,37.803455,15.443234,0.318182,28.568750,36.694583,45.955417,339.020000
pm10_gv,1462.0,24.207882,7.691013,8.333333,18.709725,22.936632,28.533282,58.625000
pm10_sul,1451.0,20.779864,7.539263,4.172766,15.354370,19.641146,25.125401,56.539015
pm25_gv,1457.0,10.947671,3.350204,1.800000,8.625000,10.208333,12.520833,35.289474
pm25_sul,1462.0,5.743247,2.731668,1.498056,3.784722,5.196135,6.890375,19.564083


In [6]:
# Time-series (outcomes)
for y in OUTCOMES:
    ax = df.plot(x="date", y=y, figsize=(8,3), legend=False, title=f"Daily {y}")
    ax.set_xlabel("")
    plt.tight_layout()
    plt.savefig(f"/content/outputs/ts_{y}.png")
    plt.close()

# Time-series (pollutants)
for p in POLLUTANTS:
    ax = df.plot(x="date", y=p, figsize=(8,3), legend=False, title=f"Daily {p}")
    ax.set_xlabel("")
    plt.tight_layout()
    plt.savefig(f"/content/outputs/ts_{p}.png")
    plt.close()


In [7]:
# Pollutant correlation (matrix + heatmap)
if len(POLLUTANTS) >= 2:
    cm = df[POLLUTANTS].corr(method="pearson")
    plt.figure(figsize=(6,5))
    sns.heatmap(cm, vmin=-1, vmax=1, cmap="coolwarm", square=True, cbar_kws={"shrink":0.8})
    plt.title("Pollutant correlation")
    plt.tight_layout()
    plt.savefig("/content/outputs/corr_pollutants.png")
    plt.close()
    cm.to_csv("/content/outputs/corr_pollutants.csv")


In [7]:
# Table 2: Outcome–Pollutant correlations (same-day)
records = []
for y in OUTCOMES:
    for p in POLLUTANTS:
        r = df[[y, p]].corr().iloc[0,1]
        records.append({"outcome": y, "pollutant": p, "r": r})
corr_y_p = pd.DataFrame(records).sort_values(["outcome","r"], ascending=[True, False])
corr_y_p.to_csv("/content/outputs/corr_outcome_pollutant.csv", index=False)
corr_y_p.head(10)


In [8]:
def iqr_scale(series: pd.Series):
    """Center by median and scale by IQR; return z, IQR, median."""
    q75, q25 = np.nanpercentile(series, [75, 25])
    iqr = q75 - q25
    med = np.nanmedian(series)
    if iqr == 0 or np.isnan(iqr):
        return (series - med), 1.0, med
    return (series - med) / iqr, iqr, med

def overdispersion_glm_poisson(result):
    """Pearson chi2 / df_resid."""
    rp = result.resid_pearson
    return (np.sum(rp**2) / result.df_resid) if result.df_resid > 0 else np.nan

def match_region_pollutants(outcome, pollutants):
    """Prefer pollutants with same region suffix as the outcome, if any."""
    m = re.search(r"_(\w+)$", outcome)
    if m:
        suf = m.group(1)
        matched = [p for p in pollutants if p.endswith("_"+suf)]
        if matched:
            return matched
    return pollutants


In [10]:
FAST_MODE = True       # True = faster; False = all pollutants & lags 0..3
USE_NB_IF_OVERDISP = True  # Fit Negative Binomial if Poisson overdispersion > 1.2
DF_PER_YEAR = 7        # spline degrees per year (reduce to 4 for speed)
MAX_LAG = 1 if FAST_MODE else 3

# Table 2: Outcome–Pollutant correlations (same-day)
records = []
for y in OUTCOMES:
    for p in POLLUTANTS:
        r = df[[y, p]].corr().iloc[0,1]
        records.append({"outcome": y, "pollutant": p, "r": r})
corr_y_p = pd.DataFrame(records).sort_values(["outcome","r"], ascending=[True, False])
corr_y_p.to_csv("/content/outputs/corr_outcome_pollutant.csv", index=False)

if FAST_MODE:
    # Top-2 pollutants per outcome by absolute correlation
    top_corr = (corr_y_p
                .assign(abs_r=lambda d: d["r"].abs())
                .sort_values(["outcome","abs_r"], ascending=[True, False])
                .groupby("outcome")
                .head(2)
                .reset_index(drop=True))
    WORK_PAIRS = top_corr[["outcome","pollutant"]].drop_duplicates().values.tolist()
else:
    # All outcome × matched pollutants
    WORK_PAIRS = []
    for y in OUTCOMES:
        for p in match_region_pollutants(y, POLLUTANTS):
            WORK_PAIRS.append([y,p])

len(WORK_PAIRS), WORK_PAIRS[:6]

(16,
 [['sih_gv_i_kid', 'no2_gv'],
  ['sih_gv_i_kid', 'co_gv'],
  ['sih_gv_i_old', 'no2_gv'],
  ['sih_gv_i_old', 'co_gv'],
  ['sih_gv_j_kid', 'co_gv'],
  ['sih_gv_j_kid', 'no2_gv']])

In [11]:
def fit_models_for(y, p, lag=0, df_per_year=7, use_nb_if_overdisp=True):
    dat = df[[y, p, "date", "dow", "year", "t"]].copy()
    # apply lag
    if lag > 0:
        dat[p] = dat[p].shift(lag)
    dat = dat.dropna(subset=[y, p, "t", "dow"])

    # scale exposure by IQR
    dat["z"], iqr, med = iqr_scale(dat[p])

    # spline df across total years
    years = dat["year"].max() - dat["year"].min() + 1
    df_time = max(4, int(df_per_year * years))

    # Poisson GLM with robust SEs: outcome ~ z + dow + bs(t)
    formula = f"Q('{y}') ~ z + C(dow) + bs(t, df={df_time})"
    glm_pois = smf.glm(formula=formula, data=dat, family=sm.families.Poisson()).fit(cov_type="HC0")

    # effect per IQR
    b  = glm_pois.params.get("z", np.nan)
    se = glm_pois.bse.get("z", np.nan)
    disp = overdispersion_glm_poisson(glm_pois)
    pct = (np.exp(b) - 1) * 100
    lcl, ucl = (np.exp(b - 1.96*se) - 1) * 100, (np.exp(b + 1.96*se) - 1) * 100

    out = [{
        "model":"Poisson_robust","outcome":y,"pollutant":p,"lag":lag,"df_time":df_time,
        "beta":b,"se":se,"p":glm_pois.pvalues.get("z", np.nan),
        "overdispersion":disp,"iqr":iqr,"pct_change":pct,"lcl":lcl,"ucl":ucl,
        "n":int(glm_pois.nobs)
    }]

    # Optional Negative Binomial (NB2) if overdispersed
    if use_nb_if_overdisp and (disp is not None) and np.isfinite(disp) and disp > 1.2:
        y_mat, X_mat = dmatrices(formula, data=dat, return_type="dataframe")
        nb2 = NB2(y_mat, X_mat).fit(disp=0)
        if "z" in X_mat.columns:
            b_nb  = nb2.params["z"]
            se_nb = nb2.bse["z"]
            pct_nb = (np.exp(b_nb) - 1) * 100
            lcl_nb, ucl_nb = (np.exp(b_nb - 1.96*se_nb) - 1) * 100, (np.exp(b_nb + 1.96*se_nb) - 1) * 100
            out.append({
                "model":"NegBin_NB2","outcome":y,"pollutant":p,"lag":lag,"df_time":df_time,
                "beta":b_nb,"se":se_nb,"p":nb2.pvalues["z"],
                "overdispersion":disp,"iqr":iqr,"pct_change":pct_nb,"lcl":lcl_nb,"ucl":ucl_nb,
                "n":int(y_mat.shape[0])
            })

    return pd.DataFrame(out)


In [12]:
results = []
for y, p in tqdm(WORK_PAIRS, desc="Outcome×Pollutant"):
    for L in range(0, MAX_LAG+1):
        try:
            res = fit_models_for(y, p, lag=L, df_per_year=DF_PER_YEAR, use_nb_if_overdisp=USE_NB_IF_OVERDISP)
            results.append(res)
        except Exception as e:
            print(f"[WARN] {y} ~ {p} (lag {L}) failed:", e)

res_all = pd.concat(results, ignore_index=True) if results else pd.DataFrame()
res_all.to_csv("/content/outputs/models_all.csv", index=False)
res_all.head(10)


Outcome×Pollutant:  25%|██▌       | 4/16 [00:01<00:06,  2.00it/s]/usr/local/lib/python3.12/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.12/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
Outcome×Pollutant:  31%|███▏      | 5/16 [00:02<00:05,  2.11it/s]/usr/local/lib/python3.12/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.12/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimiz

[WARN] sih_sul_i_kid ~ o3_gv (lag 0) failed: NaN, inf or invalid value detected in weights, estimation infeasible.
[WARN] sih_sul_i_kid ~ o3_gv (lag 1) failed: NaN, inf or invalid value detected in weights, estimation infeasible.


Outcome×Pollutant: 100%|██████████| 16/16 [00:04<00:00,  3.83it/s]


,model,outcome,pollutant,lag,df_time,beta,se,p,overdispersion,iqr,pct_change,lcl,ucl,n
0,Poisson_robust,sih_gv_i_kid,no2_gv,0,35,0.115335,0.103610,0.265634,1.051763,9.756059,12.224972,-8.400033,37.493983,1451
1,Poisson_robust,sih_gv_i_kid,no2_gv,1,35,0.235918,0.097024,0.015034,1.049164,9.737561,26.607078,4.681461,53.125032,1450
2,Poisson_robust,sih_gv_i_kid,co_gv,0,35,0.083714,0.071048,0.238693,1.057955,114.356334,8.731740,-5.402719,24.978131,1462
3,Poisson_robust,sih_gv_i_kid,co_gv,1,35,0.076205,0.073027,0.296706,1.058530,114.379533,7.918366,-6.473671,24.525080,1461
4,Poisson_robust,sih_gv_i_old,no2_gv,0,35,-0.004585,0.019784,0.816732,1.337986,9.756059,-0.457446,-4.243501,3.478304,1451
5,NegBin_NB2,sih_gv_i_old,no2_gv,0,35,-0.006380,0.020016,0.749926,1.337986,9.756059,-0.635941,-4.458580,3.339644,1451
6,Poisson_robust,sih_gv_i_old,no2_gv,1,35,-0.033396,0.020383,0.101336,1.335059,9.737561,-3.284420,-7.072106,0.657649,1450
7,NegBin_NB2,sih_gv_i_old,no2_gv,1,35,-0.034434,0.020218,0.088543,1.335059,9.737561,-3.384817,-7.138579,0.520684,1450
8,Poisson_robust,sih_gv_i_old,co_gv,0,35,0.019278,0.014289,0.177281,1.328806,114.356334,1.946469,-0.868982,4.841883,1462
9,NegBin_NB2,sih_gv_i_old,co_gv,0,35,0.019233,0.013475,0.153505,1.328806,114.356334,1.941911,-0.715319,4.670259,1462


In [13]:
# Primary table (lag 0 only) — Table 3 in your report
tbl0 = (res_all.query("lag == 0")
        .sort_values(["outcome","model","pct_change"], ascending=[True, True, False]))
tbl0.to_csv("/content/outputs/primary_models_lag0.csv", index=False)
tbl0.head(15)


,model,outcome,pollutant,lag,df_time,beta,se,p,overdispersion,iqr,pct_change,lcl,ucl,n
0,Poisson_robust,sih_gv_i_kid,no2_gv,0,35,0.115335,0.103610,0.265634,1.051763,9.756059,12.224972,-8.400033,37.493983,1451
2,Poisson_robust,sih_gv_i_kid,co_gv,0,35,0.083714,0.071048,0.238693,1.057955,114.356334,8.731740,-5.402719,24.978131,1462
9,NegBin_NB2,sih_gv_i_old,co_gv,0,35,0.019233,0.013475,0.153505,1.328806,114.356334,1.941911,-0.715319,4.670259,1462
5,NegBin_NB2,sih_gv_i_old,no2_gv,0,35,-0.006380,0.020016,0.749926,1.337986,9.756059,-0.635941,-4.458580,3.339644,1451
8,Poisson_robust,sih_gv_i_old,co_gv,0,35,0.019278,0.014289,0.177281,1.328806,114.356334,1.946469,-0.868982,4.841883,1462
4,Poisson_robust,sih_gv_i_old,no2_gv,0,35,-0.004585,0.019784,0.816732,1.337986,9.756059,-0.457446,-4.243501,3.478304,1451
13,NegBin_NB2,sih_gv_j_kid,co_gv,0,35,0.032036,0.016201,0.047997,1.326979,114.356334,3.255486,0.028190,6.586907,1462
17,NegBin_NB2,sih_gv_j_kid,no2_gv,0,35,0.006506,0.024038,0.786673,1.335812,9.756059,0.652675,-3.979590,5.508412,1451
12,Poisson_robust,sih_gv_j_kid,co_gv,0,35,0.032028,0.015536,0.039252,1.326979,114.356334,3.254657,0.157876,6.447188,1462
16,Poisson_robust,sih_gv_j_kid,no2_gv,0,35,0.008575,0.024069,0.721647,1.335812,9.756059,0.861148,-3.786428,5.733224,1451


In [14]:
# Choose top pollutant per outcome (Poisson_robust) and plot lag response
top_per_outcome = (
    res_all[res_all["model"]=="Poisson_robust"]
           .sort_values(["outcome","pct_change"], ascending=[True, False])
           .groupby("outcome", as_index=False)
           .first()[["outcome","pollutant"]]
)

for _, row in top_per_outcome.iterrows():
    y, p = row["outcome"], row["pollutant"]
    prof = (res_all[(res_all["outcome"]==y) & (res_all["pollutant"]==p) & (res_all["model"]=="Poisson_robust")]
            .sort_values("lag"))
    if prof.empty:
        continue
    plt.figure(figsize=(6,4))
    plt.axhline(0, ls="--")
    plt.errorbar(prof["lag"], prof["pct_change"],
                 yerr=[prof["pct_change"]-prof["lcl"], prof["ucl"]-prof["pct_change"]],
                 fmt="-o")
    plt.title(f"Lag response: {y} vs {p} (per IQR)")
    plt.xlabel("Lag (days)")
    plt.ylabel("% change in admissions")
    plt.tight_layout()
    plt.savefig(f"/content/outputs/lag_profile_{y}_{p}.png")
    plt.close()


In [15]:
def rnd(x):
    return np.round(x, 3) if pd.notna(x) else x

pretty0 = tbl0.copy()
for c in ["beta","se","p","overdispersion","pct_change","lcl","ucl"]:
    if c in pretty0.columns:
        pretty0[c] = pretty0[c].apply(rnd)

print("\nTop finding per outcome (Poisson_robust, lag 0):\n")
view = (
    pretty0[pretty0["model"]=="Poisson_robust"]
    .sort_values(["outcome","pct_change"], ascending=[True, False])
    .groupby("outcome", as_index=False)
    .first()[["outcome","pollutant","pct_change","lcl","ucl","p","overdispersion","n"]]
)
print(view.to_string(index=False))

print("\nFiles saved in /content/outputs:\n",
      "- descriptive_stats.csv (Table 1)\n",
      "- corr_outcome_pollutant.csv (Table 2)\n",
      "- primary_models_lag0.csv (Table 3)\n",
      "- corr_pollutants.png (Figure 3)\n",
      "- ts_*.png for outcomes and pollutants (Figures 1–2)\n",
      "- lag_profile_*.png (Figure 4)\n")



Top finding per outcome (Poisson_robust, lag 0):

      outcome pollutant  pct_change     lcl    ucl     p  overdispersion    n
 sih_gv_i_kid    no2_gv      12.225  -8.400 37.494 0.266           1.052 1451
 sih_gv_i_old     co_gv       1.946  -0.869  4.842 0.177           1.329 1462
 sih_gv_j_kid     co_gv       3.255   0.158  6.447 0.039           1.327 1462
 sih_gv_j_old   no2_sul       6.796   2.435 11.343 0.002           1.091 1459
sih_sul_i_kid    no2_gv      11.786 -29.600 77.502 0.637           0.907 1451
sih_sul_i_old   no2_sul      15.584   7.528 24.243 0.000           0.973 1459
sih_sul_j_kid     co_gv       2.425  -4.199  9.507 0.482           1.105 1462
sih_sul_j_old   pm10_gv       3.134  -7.957 15.562 0.595           1.018 1462

Files saved in /content/outputs:
 - descriptive_stats.csv (Table 1)
 - corr_outcome_pollutant.csv (Table 2)
 - primary_models_lag0.csv (Table 3)
 - corr_pollutants.png (Figure 3)
 - ts_*.png for outcomes and pollutants (Figures 1–2)
 - lag_profil

In [16]:
import shutil
shutil.make_archive("/content/outputs_zip", "zip", "/content/outputs")

from google.colab import files
files.download("/content/outputs_zip.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>